In [ ]:
!pip install --upgrade openai
import pandas as pd
import time
from google.colab import files

# Upload your CSV file
# uploaded = files.upload()  # Upload: sarcasm_balanced_5000_with_rephrased.csv


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
# Upload your CSV file
uploaded = files.upload()  # Upload: sarcasm_balanced_5000_with_rephrased.csv
df = pd.read_csv("sarcasm_5000_gpt_cognitive_updated.csv")

# Ensure all necessary columns exist
for col in [
    "rephrased_comment",
    "cognitive_explanation",
    "textual_explanation",
    "contrastive_explanation",
    "rule_based_explanation"
]:
    if col not in df.columns:
        df[col] = ""

# Select sarcastic comments that haven't been fully processed
mask_unrephrased = (df["label"] == 1) & (
    df["rephrased_comment"].isna() | df["rephrased_comment"].eq("") | df["rephrased_comment"].eq("[ERROR]") |
    df["textual_explanation"].eq("") | df["contrastive_explanation"].eq("") | df["rule_based_explanation"].eq("")
)

to_process = df[mask_unrephrased].head(3000)  # adjust daily batch size

def rephrase_and_explain_full(comment):
    prompt = (
        f"Rewrite the following sarcastic comment as a sincere, literal statement.\n"
        f"Then explain it from multiple perspectives for human understanding.\n\n"
        f"Comment: \"{comment}\"\n\n"
        f"Respond in this exact format:\n"
        f"Rephrased: <literal version>\n"
        f"Cognitive Explanation: <reason for sarcasm>\n"
        f"Textual Explanation: <plain English summary>\n"
        f"Contrastive Explanation: <compare sarcastic vs. sincere>\n"
        f"Rule-Based Explanation: <rules or cues showing sarcasm>"
    )
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
        )
        text = response.choices[0].message.content.strip()

        # Parse response
        rephrased, cog, tex, cont, rule = "[ERROR]", "[ERROR]", "[ERROR]", "[ERROR]", "[ERROR]"

        if all(x in text for x in ["Rephrased:", "Cognitive Explanation:", "Textual Explanation:", "Contrastive Explanation:", "Rule-Based Explanation:"]):
            rephrased = text.split("Rephrased:")[1].split("Cognitive Explanation:")[0].strip()
            cog = text.split("Cognitive Explanation:")[1].split("Textual Explanation:")[0].strip()
            tex = text.split("Textual Explanation:")[1].split("Contrastive Explanation:")[0].strip()
            cont = text.split("Contrastive Explanation:")[1].split("Rule-Based Explanation:")[0].strip()
            rule = text.split("Rule-Based Explanation:")[1].strip()

        return rephrased, cog, tex, cont, rule

    except Exception as e:
        print(f"Error: {e}")
        return "[ERROR]", "[ERROR]", "[ERROR]", "[ERROR]", "[ERROR]"

# Process and store in DataFrame
for i, row in to_process.iterrows():
    idx = row.name
    print(f"Processing {i+1}/500: {row['comment'][:60]}...")
    rephrased, cog, tex, cont, rule = rephrase_and_explain_full(row["comment"])
    df.at[idx, "rephrased_comment"] = rephrased
    df.at[idx, "cognitive_explanation"] = cog
    df.at[idx, "textual_explanation"] = tex
    df.at[idx, "contrastive_explanation"] = cont
    df.at[idx, "rule_based_explanation"] = rule
    time.sleep(1)

# Save updated file
output_path = "sarcasm_5000_cognitiveXAI_fivecols.csv"
df.to_csv(output_path, index=False)
files.download(output_path)


Saving sarcasm_5000_gpt_cognitive_updated.csv to sarcasm_5000_gpt_cognitive_updated.csv
Processing 2/500: She was just makin sure you didnt get kidnapped like the mov...
Processing 7/500: yeah, like the president is a big deal!...
Processing 9/500: Like sooooo yesterday...
Processing 10/500: That was the NJPW contingent...
Processing 11/500: Id rather have Rashford...
Processing 17/500: Represents the accuracy of dive bombers against American shi...
Processing 19/500: "stream of consciousness"...
Processing 21/500: Nope, if he ain't tackle him it don't count...
Processing 22/500: Funny, I thought she was Mike Dalton...
Processing 23/500: But bro look at how pretty the client is!...
Processing 25/500: No, you should just get over it...
Processing 27/500: Oh please, they are a complete nothingburrito!...
Processing 28/500: Yeah, no point in anyone playing medic now....
Processing 29/500: Isn't Canadian Christmas on a different date?...
Processing 30/500: Can you make it green instead....

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#df = pd.read_csv("sarcasm_balanced_5000_with_rephrased2.csv")
df = pd.read_csv("sarcasm_5000_gpt_cognitive_updated.csv")


# Ensure both new columns exist
if "rephrased_comment" not in df.columns:
    df["rephrased_comment"] = ""

if "cognitive_explanation" not in df.columns:
    df["cognitive_explanation"] = ""

# Select sarcastic comments that haven't been processed
mask_unrephrased = (df["label"] == 1) & (
    df["rephrased_comment"].isna() | df["rephrased_comment"].eq("") | df["rephrased_comment"].eq("[ERROR]")
)

to_process = df[mask_unrephrased].head(800)

def rephrase_and_explain(comment):
    prompt = (
        f"Rewrite the following sarcastic comment as a sincere, literal statement. "
        f"Then explain why it is sarcastic.\n\n"
        f"Comment: \"{comment}\"\n\n"
        f"Respond in the format:\n"
        f"Rephrased: <literal version>\n"
        f"Explanation: <cognitive explanation>"
    )
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
        )
        result = response.choices[0].message.content.strip()

        if "Rephrased:" in result and "Explanation:" in result:
            rephrased = result.split("Rephrased:")[1].split("Explanation:")[0].strip()
            explanation = result.split("Explanation:")[1].strip()
        else:
            rephrased = "[ERROR]"
            explanation = "[ERROR]"

        return rephrased, explanation

    except Exception as e:
        print(f"Error: {e}")
        return "[ERROR]", "[ERROR]"

for i, row in to_process.iterrows():
    idx = row.name
    print(f"Processing {i+1}/500: {row['comment'][:60]}...")
    rephrased, explanation = rephrase_and_explain(row["comment"])
    df.at[idx, "rephrased_comment"] = rephrased
    df.at[idx, "cognitive_explanation"] = explanation
    time.sleep(1)  # To avoid rate limits

output_path = "sarcasm_5000_gpt_cognitive_updated.csv"
df.to_csv(output_path, index=False)
files.download(output_path)



Processing 951/500: You said "I'm know there's more" maybe you should've drafted...
Processing 954/500: It looks like not enough people have told you yet that you c...
Processing 955/500: I wonder if this guy hangs with Newman and lights one up?...
Processing 957/500: Penoy pride gaiz!...
Processing 959/500: nice...
Processing 960/500: gr8 counter argument...
Processing 964/500: Yeah it destroys his qol to have to aim from time to time......
Processing 968/500: More heads than Schumacher ever brought....
Processing 969/500: Yeah, Dyn's security engineers are definitely shitty....
Processing 970/500: Maybe Hera has a secret evil twin that will turn good after ...
Processing 971/500: Ret paladin...
Processing 972/500: Oh wow, you came up with that yourself?...
Processing 974/500: Bamco could make the game easier...
Processing 975/500: MTC - S3RL is a BANGER...
Processing 976/500: Obviously videoshopped...
Processing 977/500: I mean, at least they had the sense to plug the A/C straight...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>